In [ ]:
import openai
import os
#from langchain_ollama import ChatOllama
#import ollama
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

In [64]:
inform_llm_of_opponent_strategy = "Your oppoent employs the follwoing strategy: "

nash = "with a K, always bet or call; with a Q, always check after you check, and call with a 1/3 probability after you bet; with a J, always fold after you bet, and bet with a probability of 1/3 after you check."
copy_player1 = "always call after you bet, and always check after you check."
random = "choose an action at random with equal probability."

#format_instruction_explain_after = "Please first output your choice of action as player 1 in the following format: Bet/Check, then explain your choice."
cot_instruction = "Think step by step:\n- What could your opponent's card be?\n- Determine the pot size and calculate the net gain or loss considering each player's contribution to the pot.\n- Evaluate the expected payoff for each strategy player 1 can adopt."
format_instruction_no_explain="Please provide your choice of action as player 1 in the following format without any explanation: Bet/Check."

def generate_prompt(llm_card,
                    inform_llm_of_opponent_strategy="",
                    strategy_description="",
                    cot_instruction="",
                    format_instruction=""):
    
    prompt = f"""
You are playing a single round of a Poker game as player 1 against an opponent (player 2). Here are the game rules:
Each player antes 1 chip. Each player is dealt a unique card from a deck consisting of only a Jack (J), a Queen (Q), and a King (K). The third card not dealt is left unseen.
Player 1 can check or bet 1 chip.
If player 1 checks, player 2 can check or bet 1 chip.
  > If player 2 checks, a showdown occurs.
  > If player 2 bets, player 1 can fold or call.
      > If player 1 folds, player 2 takes the entire pot.
      > If player 1 calls, a showdown occurs.
If player 1 bets, player 2 can fold or call.
  > If player 2 folds, player one takes the entire pot.
  > If player 2 calls, a showdown occurs.
{inform_llm_of_opponent_strategy} {strategy_description} The game starts now. Your card is {llm_card}. What is your move? {cot_instruction} {format_instruction}"""
          
    return prompt

print(generate_prompt("Q",
                      inform_llm_of_opponent_strategy="",
                      strategy_description="",
                      cot_instruction="",
                      format_instruction=""))
print("\n===========================================================================================================")
print(generate_prompt("J",
                      inform_llm_of_opponent_strategy=inform_llm_of_opponent_strategy,
                      strategy_description=nash,
                      cot_instruction="",
                      format_instruction=format_instruction_no_explain))


You are playing a single round of a Poker game as player 1 against an opponent (player 2). Here are the game rules:
Each player antes 1 chip. Each player is dealt a unique card from a deck consisting of only a Jack (J), a Queen (Q), and a King (K). The third card not dealt is left unseen.
Player 1 can check or bet 1 chip.
If player 1 checks, player 2 can check or bet 1 chip.
  > If player 2 checks, a showdown occurs.
  > If player 2 bets, player 1 can fold or call.
      > If player 1 folds, player 2 takes the entire pot.
      > If player 1 calls, a showdown occurs.
If player 1 bets, player 2 can fold or call.
  > If player 2 folds, player one takes the entire pot.
  > If player 2 calls, a showdown occurs.
  The game starts now. Your card is Q. What is your move?  


You are playing a single round of a Poker game as player 1 against an opponent (player 2). Here are the game rules:
Each player antes 1 chip. Each player is dealt a unique card from a deck consisting of only a Jack (J), 

In [65]:
#models_to_test = ["gpt-4-turbo", "gpt-4o", "o3-mini"]
#models_to_test = ["o3-mini"]

# gpt-4-turbo
    # K: bet*5
    # Q: bet*4, check*1
    # J: check*5

# gpt-4-turbo
    # K: bet*5
    # Q: bet*4, check*1
    # J: check*5

llm_card_list = ["K", "Q", "J"]
strategy_descriptions = [random, copy_player1, nash]

client =openai.OpenAI()
def get_llm_response(model, prompt, temp=0.2):
    if model[0] != "o":
        response = client.chat.completions.create(
                messages=[
                    {
                    "role": "user",
                    "content": prompt
                    }
                    ],
                    model=model,
                    temperature=temp,
                    logprobs=True
                )
    else:
        response = client.chat.completions.create(
                messages=[
                    {
                    "role": "user",
                    "content": prompt
                    }
                    ],
                    model=model,
                )    
    print(response.choices[0].message.content)
    return response

In [ ]:
os.chdir("/log-files/exp-2")

In [ ]:
file_A = "LLMs-responses-A.txt"

models_to_test, temps = ["gpt-4-turbo", "gpt-4o"], [0.2, 0.8]
for model in models_to_test:
    for temp in temps:
        print(f"**Testing model {model} @ temp = {temp}**")
        with open(file_A, "a") as f:
            f.write(f"Testing model {model} @ temp = {temp}")
        for llm_card in llm_card_list:
            print(f"  < Card is {llm_card} >")
            with open(file_A, "a") as f:
                f.write(f"  \n< Card is {llm_card} >")
            prompt = generate_prompt(llm_card,
                                 inform_llm_of_opponent_strategy="",
                                 strategy_description="",
                                 cot_instruction="",
                                 format_instruction="")
            for _ in range(5):
                print(f"    Trial {_+1}:")
                response = get_llm_response(model, prompt, temp=temp)
                with open(file_A, "a") as f:
                    f.write(f"\n    Trial {_+1}:")
                    f.write(f"\n    {response.choices[0].message.content}")
        print("\n===========================================================================================================") 
        with open(file_A, "a") as f:
            f.write("\n===============\n")
f.close()

In [ ]:
# models_to_test = ["o3-mini", "o3", "o1", "o4-mini"]
models_to_test = ["o3-mini"]
for model in models_to_test:
    print(f"**Testing model {model}**")
    with open(file_A, "a") as f:
        f.write(f"Testing model {model}")
    for llm_card in llm_card_list:
        print(f"  < Card is {llm_card} >")
        with open(file_A, "a") as f:
            f.write(f"  \n< Card is {llm_card} >")
        prompt = generate_prompt(llm_card,
                                 inform_llm_of_opponent_strategy="",
                                 strategy_description="",
                                 cot_instruction="",
                                 format_instruction="")
        for _ in range(5):
            print(f"    Trial {_+1}:")
            response = get_llm_response(model, prompt)
            with open(file_A, "a") as f:
                f.write(f"\n    Trial {_+1}:")
                f.write(f"\n    {response.choices[0].message.content}")
    print("\n===========================================================================================================") 
    with open(file_A, "a") as f:
        f.write("\n===============\n")
f.close()

In [ ]:
file_B = "LLMs-responses-B.txt"

models_to_test = ["gpt-4-turbo", "gpt-4o"]
for model in models_to_test:
    for temp in temps:
        print(f"**Testing model {model} @ temp = {temp}**")
        with open(file_B, "a") as f:
            f.write(f"Testing model {model} @ temp = {temp}")
        for llm_card in llm_card_list:
            print(f"  < Card is {llm_card} >")
            with open(file_B, "a") as f:
                f.write(f"  \n< Card is {llm_card} >")
            for strategy in strategy_descriptions:
                print(f"    < Strategy: {strategy} >")
                with open(file_B, "a") as f:
                    f.write(f"\n    < Strategy: {strategy} >")          
                prompt = generate_prompt(llm_card,
                                 inform_llm_of_opponent_strategy=inform_llm_of_opponent_strategy,
                                 strategy_description=strategy,
                                 cot_instruction="",
                                 format_instruction="")
            
                response = get_llm_response(model, prompt, temp=temp)
                with open(file_B, "a") as f:
                    f.write(f"\n    {response.choices[0].message.content}")
        print("\n===========================================================================================================") 
        with open(file_B, "a") as f:
            f.write("\n===============\n")
f.close()

In [ ]:
print("**Testing model o3-mini**")
with open(file_B, "a") as f:
    f.write(f"Testing model o3-mini")
for llm_card in llm_card_list:
    print(f"  < Card is {llm_card} >")
    with open(file_B, "a") as f:
        f.write(f"  \n< Card is {llm_card} >")
    for strategy in strategy_descriptions:
        print(f"    < Strategy: {strategy} >")
        with open(file_B, "a") as f:
            f.write(f"\n    < Strategy: {strategy} >")   
        prompt = generate_prompt(llm_card,
                                     inform_llm_of_opponent_strategy=inform_llm_of_opponent_strategy,
                                     strategy_description=strategy)
        response = get_llm_response("o3-mini", prompt)
        with open(file_B, "a") as f:
            f.write(f"\n    {response.choices[0].message.content}")
print("\n===========================================================================================================") 
with open(file_B, "a") as f:
    f.write("\n===============\n")
f.close()

In [ ]:
file_C = "LLMs-responses-C.txt"

models_to_test = ["gpt-4-turbo", "gpt-4o"]
for model in models_to_test:
    for temp in temps:
        print(f"**Testing model {model} @ temp = {temp}**")
        with open(file_C, "a") as f:
            f.write(f"Testing model {model} @ temp = {temp}")
        for llm_card in llm_card_list:
            print(f"  < Card is {llm_card} >")
            with open(file_C, "a") as f:
                f.write(f"  \n< Card is {llm_card} >")
            for strategy in strategy_descriptions:
                print(f"    < Strategy: {strategy} >")
                with open(file_C, "a") as f:
                    f.write(f"\n    < Strategy: {strategy} >")          
                prompt = generate_prompt(llm_card,
                                 inform_llm_of_opponent_strategy=inform_llm_of_opponent_strategy,
                                 strategy_description=strategy,
                                 cot_instruction=cot_instruction,
                                 format_instruction="")
            
                response = get_llm_response(model, prompt, temp=temp)
                with open(file_C, "a") as f:
                    f.write(f"\n    {response.choices[0].message.content}")
        print("\n===========================================================================================================") 
        with open(file_C, "a") as f:
            f.write("\n===============\n")
f.close()